<a href="https://colab.research.google.com/github/FuShenChien/2nd-ML100Days/blob/master/%E3%80%8Ctw_stock_treemap_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

每日1600左右更新

In [1]:
!pip install finlab > log.txt

ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import finlab
from finlab import data
import plotly.express as px


In [3]:
finlab.login('vPrVSU1klPn98i5OBK3HQRjQBhR9odXvB6dqSOsU/GvjjkNSgQ2Q0vr7Y/kSI8Wa')
close=data.get('price:收盤價')
basic_info=data.get('company_basic_info')

## Plot

relative_market_strength為相對市場強度，以市場加權平均報酬率為color_continuous_midpoint

預設為False，為絕對漲跌幅判斷，以0為color_continuous_midpoint，漲勢越強越紅，反之越綠

In [8]:
class TwStockTreeMap:

    def __init__(self, close, basic_info, start=None, end=None):
        self.start = start
        self.end = end
        self.close = close
        self.basic_info = basic_info

    # dataframe filter by selected date
    def df_date_filter(self, df, start=None, end=None):
        if start:
            df = df[df.index >= start]
        if end:
            df = df[df.index <= end]
        return df

    # map stock_name from basic info(stock_id +name)
    def map_stock_name(self, basic_info, s):
        target = basic_info[basic_info['stock_id'].str.find(s) > -1]
        if len(target) > 0:
            s = target['stock_id'].values[0]
        return s

    def create_data(self):
        close_data = self.df_date_filter(self.close, self.start, end=self.end)
        return_ratio = (close_data.iloc[-1] / close_data.iloc[0]).dropna().replace(np.inf, 0)
        return_ratio = round((return_ratio - 1) * 100, 2)
        return_ratio = pd.concat([return_ratio, close_data.iloc[-1]], axis=1).dropna()
        return_ratio = return_ratio.reset_index()
        return_ratio.columns = ['stock_id', 'return_ratio', 'close']
        return_ratio['stock_id'] = return_ratio['stock_id'].apply(lambda s: self.map_stock_name(basic_info, s))
        return_ratio = return_ratio.merge(self.basic_info[['stock_id', '產業類別', '市場別', '實收資本額(元)']], how='left',
                                          on='stock_id')
        return_ratio = return_ratio.rename(columns={'產業類別': 'category', '市場別': 'market', '實收資本額(元)': 'base'})
        return_ratio['market_value'] = round(return_ratio['base'] / 10 * return_ratio['close'] / 100000000, 2)
        return_ratio = return_ratio.dropna(thresh=5)
        return_ratio['country'] = 'TW-Stock'
        return_ratio['return_ratio_text_info']=return_ratio['return_ratio'].astype(str).apply(lambda s: '+' + s if '-' not in s else s) + '%'
        return return_ratio

    def create_fig(self, relative_market_strength=False):
        df = self.create_data()
        if relative_market_strength is True:
            color_continuous_midpoint = np.average(df['return_ratio'], weights=df['base'])
        else:
            color_continuous_midpoint = 0
        fig = px.treemap(df, 
                         path=['country', 'market', 'category', 'stock_id'], 
                         values='market_value',
                         color='return_ratio',
                         color_continuous_scale='Tealrose',
                         color_continuous_midpoint=color_continuous_midpoint,
                         custom_data=['return_ratio_text_info','close'],
                         title=f'TW-Stock Market TreeMap({self.start}~{self.end})',
                         width=1350, 
                         height=900)
        
        fig.update_traces(textposition='middle center', 
                          textfont_size=24,
                          texttemplate= "%{label}<br>%{customdata[0]}<br>%{customdata[1]}"
                          )
        return fig

#@title 台股漲跌與市值板塊圖
start= '2021-06-01' #@param {type:"date"}
end = '2021-07-02' #@param {type:"date"}
relative_market_strength = "False" #@param ["False", "True"] {type:"raw"}

TwStockTreeMap(close,basic_info,start,end).create_fig(relative_market_strength)

# 新增區段

In [6]:
df=TwStockTreeMap(close,basic_info,start,end).create_data()
df

,stock_id,return_ratio,close,category,market,base,market_value,country,return_ratio_text_info
224,1101 台泥,16.90,50.5,水泥工業,sii,6.157440e+10,3109.51,TW-Stock,+16.9%
225,1102 亞泥,17.44,50.5,水泥工業,sii,3.367379e+10,1700.53,TW-Stock,+17.44%
226,1103 嘉泥,17.19,22.5,水泥工業,sii,7.747805e+09,174.33,TW-Stock,+17.19%
227,1104 環泥,-0.21,23.5,水泥工業,sii,6.536092e+09,153.60,TW-Stock,-0.21%
228,1108 幸福,7.57,13.5,水泥工業,sii,4.047380e+09,54.64,TW-Stock,+7.57%
...,...,...,...,...,...,...,...,...,...
1939,9951 皇田,-3.95,92.3,電機機械,otc,7.490000e+08,69.13,TW-Stock,-3.95%
1940,9955 佳龍,-8.43,19.0,其他,sii,1.032082e+09,19.61,TW-Stock,-8.43%
1941,9958 世紀鋼,21.92,133.5,鋼鐵工業,sii,2.301021e+09,307.19,TW-Stock,+21.92%
1942,9960 邁達康,-20.52,27.3,貿易百貨,otc,3.359250e+08,9.17,TW-Stock,-20.52%
